In [3]:
##BAD APPROACH
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')

In [20]:
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Utils.utils import create_URM, create_ICM_with_events,create_ICM, combine_matrices,create_submission, write_submission
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from skopt.space import Real
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [5]:
##CREATE URM,ICM, ICM STACKED COMPLETELY TO TRAIN THE SINGLE RECOMMENDERS, AND STACKED PARTIALLY TO TUNE THE HYBRID
URM = create_URM()
ICM = create_ICM_with_events()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)
ICM_combined = combine_matrices(ICM=ICM,URM=URM)

EvaluatorHoldout: Ignoring 13637 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13625 ( 0.2%) Users that have less than 1 test interactions


In [6]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=ICM_combined)
RP3Beta_recommender.fit(topK=93, alpha=0.5423746227005897, beta=0.5795196737709303, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 10245 ( 2.8%) users with no interactions.


In [7]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=ICM_combined)
IALS_recommender.fit(epochs=50, num_factors=40, alpha=0.7917156202136513, reg=1.0)

IALSRecommenderLinear: URM Detected 10245 ( 2.8%) users with no interactions.
IALSRecommenderLinear: Epoch 1 of 50. Elapsed time 1.17 min
IALSRecommenderLinear: Epoch 2 of 50. Elapsed time 1.94 min
IALSRecommenderLinear: Epoch 3 of 50. Elapsed time 2.68 min
IALSRecommenderLinear: Epoch 4 of 50. Elapsed time 3.41 min
IALSRecommenderLinear: Epoch 5 of 50. Elapsed time 4.15 min
IALSRecommenderLinear: Epoch 6 of 50. Elapsed time 4.94 min
IALSRecommenderLinear: Epoch 7 of 50. Elapsed time 5.73 min
IALSRecommenderLinear: Epoch 8 of 50. Elapsed time 6.48 min
IALSRecommenderLinear: Epoch 9 of 50. Elapsed time 7.24 min
IALSRecommenderLinear: Epoch 10 of 50. Elapsed time 7.99 min
IALSRecommenderLinear: Epoch 11 of 50. Elapsed time 9.11 min
IALSRecommenderLinear: Epoch 12 of 50. Elapsed time 9.85 min
IALSRecommenderLinear: Epoch 13 of 50. Elapsed time 10.70 min
IALSRecommenderLinear: Epoch 14 of 50. Elapsed time 11.44 min
IALSRecommenderLinear: Epoch 15 of 50. Elapsed time 12.18 min
IALSRecommend

In [8]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=ICM_combined)
SLIME_recommender.fit(topK= 367, l1_ratio = 0.001, alpha = 0.0051965845279935575, workers=10 )

SLIMElasticNetRecommender: URM Detected 10245 ( 2.8%) users with no interactions.


100%|████████████████████████████████████▉| 18056/18059 [17:40<00:00, 31.12it/s]

In [9]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0),
    "gamma": Real(0.0, 1.0)
}
recommenders = [SLIME_recommender,RP3Beta_recommender, IALS_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[combined_matrices, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeThreeModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeThreeModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.06136886631082729, 'beta': 0.8813973165616958, 'gamma': 0.14779605487658667}
MergeThreeModelsByScore: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13625 (100.0%) in 22.77 sec. Users per second: 598
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.06136886631082729, 'beta': 0.8813973165616958, 'gamma': 0.14779605487658667} - results: PRECISION: 0.2174679, PRECISION_RECALL_MIN_DEN: 0.2237071, RECALL: 0.0641292, MAP: 0.1020529, MAP_MIN_DEN: 0.1055277, MRR: 0.4278263, NDCG: 0.2255550, F1: 0.0990496, HIT_RATE: 0.8926972, ARHR_ALL_HITS: 0.6595076, NOVELTY: 0.0056014, AVERAGE_POPULARITY: 0.5027185, DIVERSITY_MEAN_INTER_LIST: 0.8540310, DIVERSITY_HERFINDAHL: 0.9853968, COVERAGE_ITEM: 0.1191096, COVERAGE_ITEM_CORRECT: 0.0707680, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.8910623, DIVERSITY_GINI: 0.0106033, SHANNON_ENTROPY: 

100%|████████████████████████████████████▉| 18056/18059 [20:50<00:00, 14.44it/s]

SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.06136886631082729, 'beta': 0.8813973165616958, 'gamma': 0.14779605487658667} - results:
CUTOFF: 10 - PRECISION: 0.2553641, PRECISION_RECALL_MIN_DEN: 0.2603357, RECALL: 0.0635436, MAP: 0.1282185, MAP_MIN_DEN: 0.1310525, MRR: 0.4800364, NDCG: 0.2645391, F1: 0.1017646, HIT_RATE: 0.9254235, ARHR_ALL_HITS: 0.7808922, NOVELTY: 0.0056015, AVERAGE_POPULARITY: 0.5026975, DIVERSITY_MEAN_INTER_LIST: 0.8540593, DIVERSITY_HERFINDAHL: 0.9853997, COVERAGE_ITEM: 0.1187773, COVERAGE_ITEM_CORRECT: 0.0753087, COVERAGE_USER: 0.9990476, COVERAGE_USER_CORRECT: 0.9245421, DIVERSITY_GINI: 0.0105926, SHANNON_ENTROPY: 7.4182466, RATIO_DIVERSITY_HERFINDAHL: 0.9857591, RATIO_DIVERSITY_GINI: 0.0395639, RATIO_SHANNON_ENTROPY: 0.5938947, RATIO_AVERAGE_POPULARITY: 3.1707974, RATIO_NOVELTY: 0.7110453, 


SearchBayesianSkopt: Saving model in logs/MergeThreeModelsByScore

MergeThreeModelsByScore: Saving model in file 'logs/MergeThreeModelsByS

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 45.6773
Function value obtained: -0.1021
Current minimum: -0.1021
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6167595790515932, 'beta': 0.8943393751777883, 'gamma': 0.7273883912859666}
MergeThreeModelsByScore: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13625 (100.0%) in 22.46 sec. Users per second: 607
SearchBayesianSkopt: New best config found. Config 1: {'alpha': 0.6167595790515932, 'beta': 0.8943393751777883, 'gamma': 0.7273883912859666} - results: PRECISION: 0.2311780, PRECISION_RECALL_MIN_DEN: 0.2361594, RECALL: 0.0669771, MAP: 0.1099632, MAP_MIN_DEN: 0.1122209, MRR: 0.4412028, NDCG: 0.2375106, F1: 0.1038629, HIT_RATE: 0.9078899, ARHR_ALL_HITS: 0.6953859, NOVELTY: 0.0055252, AVERAGE_POPULARITY: 0.5106527, DIVERSITY_MEAN_INTER_LIST: 0.8829882, DIVERSITY_HERFINDAHL: 0.9882923, COVERAGE_ITEM: 0.0697159, COVERAGE

EvaluatorHoldout: Processed 13625 (100.0%) in 22.44 sec. Users per second: 607
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'alpha': 0.159607272422057, 'beta': 0.21404841475643283, 'gamma': 0.02862774590740947} - results: PRECISION: 0.2247193, PRECISION_RECALL_MIN_DEN: 0.2300404, RECALL: 0.0654384, MAP: 0.1044170, MAP_MIN_DEN: 0.1069135, MRR: 0.4211428, NDCG: 0.2288742, F1: 0.1013605, HIT_RATE: 0.9023853, ARHR_ALL_HITS: 0.6625765, NOVELTY: 0.0055308, AVERAGE_POPULARITY: 0.5169179, DIVERSITY_MEAN_INTER_LIST: 0.8626054, DIVERSITY_HERFINDAHL: 0.9862542, COVERAGE_ITEM: 0.0873249, COVERAGE_ITEM_CORRECT: 0.0569245, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.9007326, DIVERSITY_GINI: 0.0092012, SHANNON_ENTROPY: 7.3951535, RATIO_DIVERSITY_HERFINDAHL: 0.9866139, RATIO_DIVERSITY_GINI: 0.0343672, RATIO_SHANNON_ENTROPY: 0.5920459, RATIO_AVERAGE_POPULARITY: 3.2604938, RATIO_NOVELTY: 0.7020712, 

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 22.6047
Function val

...

EvaluatorHoldout: Processed 13625 (100.0%) in 23.06 sec. Users per second: 591
SearchBayesianSkopt: Config 168 is suboptimal. Config: {'alpha': 0.8014426797053485, 'beta': 0.1408292287396802, 'gamma': 0.6036190797478146} - results: PRECISION: 0.2379303, PRECISION_RECALL_MIN_DEN: 0.2418182, RECALL: 0.0674379, MAP: 0.1142309, MAP_MIN_DEN: 0.1158185, MRR: 0.4475136, NDCG: 0.2432337, F1: 0.1050897, HIT_RATE: 0.9119266, ARHR_ALL_HITS: 0.7133373, NOVELTY: 0.0055152, AVERAGE_POPULARITY: 0.5045922, DIVERSITY_MEAN_INTER_LIST: 0.9059402, DIVERSITY_HERFINDAHL: 0.9905874, COVERAGE_ITEM: 0.0585857, COVERAGE_ITEM_CORRECT: 0.0426934, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.9102564, DIVERSITY_GINI: 0.0099397, SHANNON_ENTROPY: 7.7480091, RATIO_DIVERSITY_HERFINDAHL: 0.9909487, RATIO_DIVERSITY_GINI: 0.0371254, RATIO_SHANNON_ENTROPY: 0.6202950, RATIO_AVERAGE_POPULARITY: 3.1827487, RATIO_NOVELTY: 0.7000959, 

Iteration No: 169 ended. Search finished for the next optimal point.
Time taken: 25.428

Iteration No: 175 ended. Search finished for the next optimal point.
Time taken: 25.6330
Function value obtained: -0.1135
Current minimum: -0.1185
Iteration No: 176 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.05919907367100235, 'beta': 0.0, 'gamma': 0.46220432948229967}
MergeThreeModelsByScore: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13625 (100.0%) in 23.29 sec. Users per second: 585
SearchBayesianSkopt: Config 175 is suboptimal. Config: {'alpha': 0.05919907367100235, 'beta': 0.0, 'gamma': 0.46220432948229967} - results: PRECISION: 0.2390459, PRECISION_RECALL_MIN_DEN: 0.2435167, RECALL: 0.0690231, MAP: 0.1183665, MAP_MIN_DEN: 0.1204929, MRR: 0.4720784, NDCG: 0.2497378, F1: 0.1071168, HIT_RATE: 0.9142752, ARHR_ALL_HITS: 0.7446804, NOVELTY: 0.0055604, AVERAGE_POPULARITY: 0.4785442, DIVERSITY_MEAN_INTER_LIST: 0.9310521, DIVERSITY_HERFINDAHL: 0.9930984, COVERAGE_ITEM: 0.0551526, COVERAGE_ITEM_COR

EvaluatorHoldout: Processed 13625 (100.0%) in 22.80 sec. Users per second: 598
SearchBayesianSkopt: Config 181 is suboptimal. Config: {'alpha': 0.04083089872731672, 'beta': 0.27523314405901006, 'gamma': 0.3795890263112832} - results: PRECISION: 0.2334752, PRECISION_RECALL_MIN_DEN: 0.2388151, RECALL: 0.0681622, MAP: 0.1130088, MAP_MIN_DEN: 0.1155829, MRR: 0.4574129, NDCG: 0.2425995, F1: 0.1055186, HIT_RATE: 0.9104587, ARHR_ALL_HITS: 0.7168391, NOVELTY: 0.0055445, AVERAGE_POPULARITY: 0.4994371, DIVERSITY_MEAN_INTER_LIST: 0.8973684, DIVERSITY_HERFINDAHL: 0.9897303, COVERAGE_ITEM: 0.0649538, COVERAGE_ITEM_CORRECT: 0.0492275, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.9087912, DIVERSITY_GINI: 0.0104986, SHANNON_ENTROPY: 7.7442947, RATIO_DIVERSITY_HERFINDAHL: 0.9900912, RATIO_DIVERSITY_GINI: 0.0392128, RATIO_SHANNON_ENTROPY: 0.6199976, RATIO_AVERAGE_POPULARITY: 3.1502323, RATIO_NOVELTY: 0.7038092, 

Iteration No: 182 ended. Search finished for the next optimal point.
Time taken: 25.4

Iteration No: 188 ended. Search finished for the next optimal point.
Time taken: 25.8207
Function value obtained: -0.1185
Current minimum: -0.1185
Iteration No: 189 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6275876957868914, 'beta': 0.8269612679708549, 'gamma': 0.9308262304624291}
MergeThreeModelsByScore: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13625 (100.0%) in 23.13 sec. Users per second: 589
SearchBayesianSkopt: Config 188 is suboptimal. Config: {'alpha': 0.6275876957868914, 'beta': 0.8269612679708549, 'gamma': 0.9308262304624291} - results: PRECISION: 0.2329761, PRECISION_RECALL_MIN_DEN: 0.2377313, RECALL: 0.0672892, MAP: 0.1114753, MAP_MIN_DEN: 0.1135534, MRR: 0.4467938, NDCG: 0.2397465, F1: 0.1044195, HIT_RATE: 0.9106055, ARHR_ALL_HITS: 0.7042854, NOVELTY: 0.0055259, AVERAGE_POPULARITY: 0.5077274, DIVERSITY_MEAN_INTER_LIST: 0.8903049, DIVERSITY_HERFINDAHL: 0.9890240, COVERAGE_ITEM: 0.

EvaluatorHoldout: Processed 13625 (100.0%) in 22.97 sec. Users per second: 593
SearchBayesianSkopt: Config 194 is suboptimal. Config: {'alpha': 0.07494743837124732, 'beta': 0.0, 'gamma': 0.6831913504120359} - results: PRECISION: 0.2391413, PRECISION_RECALL_MIN_DEN: 0.2436239, RECALL: 0.0690664, MAP: 0.1185253, MAP_MIN_DEN: 0.1207053, MRR: 0.4729442, NDCG: 0.2500212, F1: 0.1071785, HIT_RATE: 0.9139817, ARHR_ALL_HITS: 0.7458672, NOVELTY: 0.0055619, AVERAGE_POPULARITY: 0.4777971, DIVERSITY_MEAN_INTER_LIST: 0.9315802, DIVERSITY_HERFINDAHL: 0.9931512, COVERAGE_ITEM: 0.0550972, COVERAGE_ITEM_CORRECT: 0.0442992, COVERAGE_USER: 0.9981685, COVERAGE_USER_CORRECT: 0.9123077, DIVERSITY_GINI: 0.0121046, SHANNON_ENTROPY: 8.0956590, RATIO_DIVERSITY_HERFINDAHL: 0.9935134, RATIO_DIVERSITY_GINI: 0.0452112, RATIO_SHANNON_ENTROPY: 0.6481274, RATIO_AVERAGE_POPULARITY: 3.0137369, RATIO_NOVELTY: 0.7060255, 

Iteration No: 195 ended. Search finished for the next optimal point.
Time taken: 26.1028
Function val

In [23]:
recommender_new = MergeThreeModelsByScores(URM_train = combined_matrices,recommenders = recommenders)
recommender_new.fit(alpha =1.0, beta = 0.0, gamma=0.3023745211799622 )

MergeThreeModelsByScore: URM Detected 10245 ( 2.8%) users with no interactions.


In [16]:
result_df, _ = evaluator_test.evaluateRecommender(recommender_new)
print(result_df.loc[10])

EvaluatorHoldout: Processed 13637 (100.0%) in 22.64 sec. Users per second: 602
PRECISION                     0.281294
PRECISION_RECALL_MIN_DEN      0.284934
RECALL                        0.069466
MAP                           0.148658
MAP_MIN_DEN                   0.150282
MRR                           0.531495
NDCG                          0.293826
F1                            0.111417
HIT_RATE                      0.944489
ARHR_ALL_HITS                 0.882473
NOVELTY                       0.005563
AVERAGE_POPULARITY            0.477416
DIVERSITY_MEAN_INTER_LIST     0.931851
DIVERSITY_HERFINDAHL          0.993178
COVERAGE_ITEM                 0.055208
COVERAGE_ITEM_CORRECT         0.044908
COVERAGE_USER                 0.999048
COVERAGE_USER_CORRECT          0.94359
DIVERSITY_GINI                0.012132
SHANNON_ENTROPY               8.099664
RATIO_DIVERSITY_HERFINDAHL    0.993541
RATIO_DIVERSITY_GINI          0.045314
RATIO_SHANNON_ENTROPY         0.648448
RATIO_AVERAGE_POPULARITY

In [17]:
ICM_simple = create_ICM()
old_ICM = combine_matrices(ICM=ICM, URM= URM_train)

In [18]:
recommender = MergeThreeModelsByScores(URM_train = old_ICM,recommenders = recommenders)
recommender.fit(alpha=1.0, beta=0.0, gamma=0.3023745211799622)

MergeThreeModelsByScore: URM Detected 10245 ( 2.8%) users with no interactions.


In [19]:
result_df, _ = evaluator_test.evaluateRecommender(recommender)
print(result_df.loc[10])

EvaluatorHoldout: Processed 13637 (100.0%) in 23.20 sec. Users per second: 588
PRECISION                     0.278155
PRECISION_RECALL_MIN_DEN      0.281094
RECALL                        0.067176
MAP                             0.1423
MAP_MIN_DEN                   0.143394
MRR                           0.496831
NDCG                          0.283906
F1                            0.108217
HIT_RATE                      0.940309
ARHR_ALL_HITS                 0.837144
NOVELTY                       0.005504
AVERAGE_POPULARITY            0.508382
DIVERSITY_MEAN_INTER_LIST     0.904456
DIVERSITY_HERFINDAHL          0.990439
COVERAGE_ITEM                 0.059749
COVERAGE_ITEM_CORRECT          0.04286
COVERAGE_USER                 0.999048
COVERAGE_USER_CORRECT         0.939414
DIVERSITY_GINI                0.009568
SHANNON_ENTROPY               7.703102
RATIO_DIVERSITY_HERFINDAHL      0.9908
RATIO_DIVERSITY_GINI          0.035736
RATIO_SHANNON_ENTROPY           0.6167
RATIO_AVERAGE_POPULARITY

In [24]:
submission = create_submission(recommender_new)
write_submission(submission,"SLIME_RP3_IALS_Stacked_FullICM_hybridBeta")